<a href="https://colab.research.google.com/github/respect5716/Deep-Learning-Paper-Implementation/blob/master/05_Recommender/Neural%20Collaborative%20Filtering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Neural Collaborative Filtering

## 0. Paper

### Info
* TItle : Neural Collaborative Filtering
* Author : Xiangnan He et al.
* Publication : IW3C2 2017, [link](https://dl.acm.org/doi/pdf/10.1145/3038912.3052569)

### Summary
* neural network를 활용한 genereral collaborative filtering framework를 제시
* 내적을 neural network로 대체함을써 비선형적 representation을 학습할 수 있음

### Differences
* pre-train : True -> False

## 1. Setting

In [0]:
# Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [0]:
# Libraries
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf

In [0]:
# GPU Setting
!nvidia-smi

print(f'tensorflow version : {tf.__version__}')
print(f'available GPU list : {tf.config.list_physical_devices("GPU")}')

Sun May 31 07:56:29 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.82       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
# Hyperparameters
CONFIG = {
    'base_dir' : '/content/drive/Shared drives/Yoon/Project/Doing/Deep Learning Paper Implementation',
    'num_neg' : 4,
    'learning_rate' : 1e-3,
    'batch_size' : 128,
    'epoch_size' : 10
}

## 2. Data

In [0]:
def load_data():
    rating = pd.read_table('data/ratings.dat', sep='::', engine='python', header=None)
    rating.columns = ['userId', 'itemId', 'rating', 'timestamp']    
    rating = rating.sort_values(['userId', 'timestamp'])
    return rating

def get_neg_sample(pos, num_item, num_neg=None):
    pos = sorted(pos)
    
    sample = np.arange(0, num_item - len(pos))
    pos_adj = pos - np.arange(len(pos))
    search = np.searchsorted(pos_adj, sample, side='right')
    neg = sample + search

    if not num_neg:
        num_neg = min(CONFIG['num_neg'] * len(pos), len(neg))
    neg = np.random.choice(neg, num_neg)
    return neg

In [0]:
data_path = os.path.join(CONFIG['base_dir'], 'data/movielens_1m.zip')

In [0]:
!unzip $"{data_path}" -d '/content/data/'

Archive:  /content/drive/Shared drives/Yoon/Project/Doing/Deep Learning Paper Implementation/data/movielens_1m.zip
  inflating: /content/data/movies.dat  
  inflating: /content/data/ratings.dat  
  inflating: /content/data/README    
  inflating: /content/data/users.dat  


In [0]:
data = load_data()

In [0]:
data.head()

,userId,itemId,rating,timestamp
31,1,3186,4,978300019
22,1,1270,5,978300055
27,1,1721,4,978300055
37,1,1022,5,978300055
24,1,2340,3,978300103


In [0]:
user_dict = data['userId'].unique()
user_dict = {i:idx for idx,i in enumerate(user_dict)}
item_dict = data['itemId'].unique()
item_dict = {i:idx for idx,i in enumerate(item_dict)}

In [0]:
num_user = len(user_dict)
num_item = len(item_dict)
num_user, num_item

(6040, 3706)

In [0]:
data['userId'] = data['userId'].map(user_dict)
data['itemId'] = data['itemId'].map(item_dict)
data = data.groupby('userId')['itemId'].apply(list)
data = data.loc[data.apply(len) >= 20]

In [0]:
train_data = data.apply(lambda x : x[:-1])
test_data = data.apply(lambda x : x[-1])

In [0]:
pos_list = train_data.apply(lambda x : list(set(x)))
neg_list = pos_list.apply(lambda x : get_neg_sample(x, num_item))

In [0]:
pos_table = pos_list.explode().reset_index()
pos_table['label'] = 1
neg_table = neg_list.explode().reset_index()
neg_table['label'] = 0

In [0]:
train_table = pd.concat([pos_table, neg_table])
train_table.head()

,userId,itemId,label
0,0,0,1
1,0,1,1
2,0,2,1
3,0,3,1
4,0,4,1


In [0]:
train_user = train_table['userId'].values[:,None].astype(np.int32)
train_item = train_table['itemId'].values[:,None].astype(np.int32)
train_y = train_table['label'].values.astype(np.int32)

## 3. Model

In [0]:
def build_network(num_user, num_item):
    user_inputs = tf.keras.layers.Input((1,), dtype=np.int32)
    item_inputs = tf.keras.layers.Input((1,), dtype=np.int32)
    user_embedding = tf.keras.layers.Embedding(num_user, 32)(user_inputs)
    item_embedding = tf.keras.layers.Embedding(num_item, 32)(item_inputs)

    x = tf.keras.layers.Concatenate()([user_embedding, item_embedding])
    x = tf.keras.backend.squeeze(x, axis=1)
    x = tf.keras.layers.Dense(32, activation='relu')(x)
    x = tf.keras.layers.Dense(16, activation='relu')(x)
    x = tf.keras.layers.Dense(8, activation='relu')(x)
    outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)

    network = tf.keras.Model([user_inputs, item_inputs], outputs)
    network.compile(
        loss = 'binary_crossentropy',
        optimizer = tf.keras.optimizers.Adam(learning_rate = CONFIG['learning_rate'])
    )
    return network

In [0]:
network = build_network(num_user, num_item)
network.summary()

Model: "model_6"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_17 (InputLayer)           [(None, 1)]          0                                            
__________________________________________________________________________________________________
input_18 (InputLayer)           [(None, 1)]          0                                            
__________________________________________________________________________________________________
embedding_14 (Embedding)        (None, 1, 32)        193280      input_17[0][0]                   
__________________________________________________________________________________________________
embedding_15 (Embedding)        (None, 1, 32)        118592      input_18[0][0]                   
____________________________________________________________________________________________

## 4. Train

In [0]:
network.fit(
    [train_user, train_item], train_y,
    batch_size = CONFIG['batch_size'],
    epochs = CONFIG['epoch_size'],
)

Epoch 1/10
37622/37622 [==============================] - 182s 5ms/step - loss: 0.3445
Epoch 2/10
37622/37622 [==============================] - 175s 5ms/step - loss: 0.3014
Epoch 3/10
37622/37622 [==============================] - 171s 5ms/step - loss: 0.2840
Epoch 4/10
37622/37622 [==============================] - 166s 4ms/step - loss: 0.2738
Epoch 5/10
37622/37622 [==============================] - 168s 4ms/step - loss: 0.2667
Epoch 6/10
37622/37622 [==============================] - 166s 4ms/step - loss: 0.2615
Epoch 7/10
37622/37622 [==============================] - 169s 4ms/step - loss: 0.2574
Epoch 8/10
37622/37622 [==============================] - 171s 5ms/step - loss: 0.2541
Epoch 9/10
37622/37622 [==============================] - 167s 4ms/step - loss: 0.2513
Epoch 10/10
37622/37622 [==============================] - 167s 4ms/step - loss: 0.2490


## 5. Test

In [0]:
rank_list = []
for idx, i in enumerate(test_data):
    neg_item = get_neg_sample(pos_list[i], num_item, 100)
    item = [i] + list(neg_item)
    item = np.array(item)[:,None]
    user = [idx for _ in range(101)]
    user = np.array(user)[:,None]

    pred = network.predict((user, item))
    pred = pred[:,0]
    rank = np.argsort(np.argsort(-pred))[0]
    rank_list.append(rank)

In [0]:
hr1 = [i == 0 for i in rank_list]
hr5 = [i < 5 for i in rank_list]
hr10 = [i < 10 for i in rank_list]
print(f'HR@1 : {np.mean(hr1):.2f} | HR@5 : {np.mean(hr5):.2f} | HR@10 : {np.mean(hr10):.2f}')

HR@1 : 0.12 | HR@5 : 0.40 | HR@10 : 0.59
